# Comparing various ePodium files

In [2]:
import sys
import os
import glob

sys.path.insert(0,os.path.dirname(os.getcwd()))

In [3]:
import fnmatch
import warnings
import re
warnings.filterwarnings('ignore')

import helper_functions

import hashlib
import numpy as np
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mne

sns.set_theme(style="whitegrid")
%matplotlib inline

C:\Users\makeda\anaconda3\envs\mne-march2\lib\site-packages\mne\fixes.py:321: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(scipy.__version__) >= '1.1':
C:\Users\makeda\anaconda3\envs\mne-march2\lib\site-packages\mne\fixes.py:1134: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numba.__version__) < LooseVersion('0.40'):
C:\Users\makeda\anaconda3\envs\mne-march2\lib\site-packages\mne\fixes.py:1134: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numba.__version__) < LooseVersion('0.40'):


In [4]:
from config import PATH_RAW_DATA, PATH_METADATA, PATH_DATA_PROCESSED_ML

We have ascertained that the files for the ePodium project on Surf are distinct from the publicly published files they are supposedly the same as by using a checksum based on hashes. This notebook will explore the differences in files with the same name.

In [5]:
mother_files = glob.glob('C:/Projects/mother-eeg-data/*/Data/*.cnt')
listed_names_ddp_mother = []
for file in mother_files:
    file_name = os.path.basename(file)
    listed_names_ddp_mother.append(file_name)
print(listed_names_ddp_mother)

['015_thomas_mmn36w.cnt', '034_17_mc_mmn36_wk.cnt', '036_17_mc_mmn36_wk.cnt', '036_29_mc_mmn36_wk.cnt', '119_17_jr_mmn.cnt', '119_29_jr_mmn36_wk.cnt', '133_17_jr_mmn.cnt', '133_29_jr_mmn36_wk.cnt', '142_17_jd_mmn36_wk.cnt', '156_17_jd_mmn36_wk.cnt', '158_17_mr_mmn36.cnt', '158_29_mr_mmn36_wk.cnt', '162_17_mr_mmn36_wk.cnt', '162_17_mr_mmn36_wk.cnt', '164_17_jr_mmn36_wk.cnt', '164_29_jr_mmn36_wk.cnt', '172_17_mr_mmn36.cnt', '172_29_mr_mmn36_wk.cnt', '175_17_jd_mmn_wk.cnt', '162_17_mr_mmn36_wk.cnt', '164_17_jr_mmn36_wk.cnt', '164_29_jr_mmn36_wk.cnt', '172_17_mr_mmn36.cnt', '172_29_mr_mmn36_wk.cnt', '175_17_jd_mmn_wk.cnt', '179_29_mr_mmn36_wk.cnt', '162_17_mr_mmn36_wk.cnt', '164_17_jr_mmn36_wk.cnt', '164_29_jr_mmn36_wk.cnt', '172_17_mr_mmn36.cnt', '172_29_mr_mmn36_wk.cnt', '175_17_jd_mmn_wk.cnt', '179_29_mr_mmn36_wk.cnt', '162_17_mr_mmn36_wk.cnt', '164_17_jr_mmn36_wk.cnt', '164_29_jr_mmn36_wk.cnt', '172_17_mr_mmn36.cnt', '172_29_mr_mmn36_wk.cnt', '175_17_jd_mmn_wk.cnt', '179_29_mr_mmn36_wk

In [6]:
surf_files = glob.glob('C:/Projects/EEG_explorer/Data/*/*.cnt')
listed_names_surf = []
for file in surf_files:
    file_name = os.path.basename(file)
    listed_names_surf.append(file_name)
print(listed_names_surf)

['001_11_jc_mmn36_wk_mmn25_wk_mmn47_wk_mmn58_wk.cnt', '003_11_jc_mmn36_slp_mmn25_wk_mmn47_slp_mmn58_slp.cnt', '004_11_mc_mmn.cnt', '005_11_jc_mmn2.cnt', '007_11_jc_mmn2_36_wk.cnt', '007_11_jc_mmn36_wk_mmn25_wk.cnt', '008_11_jc_mmn.cnt', '008_11_jc_mmn_3.cnt', '008_11_jc_mmn_36_slp_mmn25_slp_mmn47_wk_2.cnt', '009_11_jc_mmn36_wk_mmn25_wk.cnt', '009_11_jc_mmn47_wk.cnt', '010_11_jc_mmn2.cnt', '010_11_jc_mmn36_wk_mmn25_wk.cnt', '011_11_jc_mmn36_wk_mmn47_wk.cnt', '014_11_mc_mmn36_wk_mmn25_wk.cnt', '015_11_jc_mmn36_slp_mmn25_slp_mmn47_slp_mmn58_slp.cnt', '016_11_jc_mmn36_wk.cnt', '017_11_jc_mmn36_mixed_mmn25_slp.cnt', '018_11_mc_mmn.cnt', '018_11_mc_mmn2.cnt', '019_11_mc_mmn36_mixed_mmn25_slp_mmn47mixed_mmn58_slp.cnt', '021_11_jc_mmn36_slp_mmn25_slp_mmn47_slp_mmn58_slp.cnt', '023_11_mc_mmn36_wk.cnt', '024_11_jc_mmn36_slp_mmn25_slp_mmn47_slp_mmn58_slp.cnt', '025_11_mc_mmn36_wk.cnt', '027_11_jc_mmn25_wk.cnt', '027_11_jc_mmn36.cnt', '027_11_jc_mmn47_wk.cnt', '028_11_mc_mmn25_slp.cnt', '028_11_mc

In [7]:
set_mother = set(listed_names_ddp_mother)
set_surf = set(listed_names_surf)
#len(set_mother.intersection(set_surf))
common_files_at_zero = set_mother.intersection(set_surf)
len(common_files_at_zero)

52

So it would appear we have 52 files with the same names. We will investigate them as we know the hash-checksum did not generate common numbers on them. To rehash:

In [8]:
def hash_it_up_right(origin_folder1):
    hash_list = []
    file_names = []
    non_suspects1 = glob.glob(os.path.join(origin_folder1, '*.cnt'))
    BUF_SIZE = 65536 
    sha256 = hashlib.sha256()
    for file in non_suspects1:
        with open(file, 'rb') as f:
            while True:
                data = f.read(BUF_SIZE)
                if not data:
                    break
                sha256.update(data)
        result = sha256.hexdigest()

        hash_list.append(result)
        file_names.append(file)
        
    df = pd.DataFrame(hash_list, file_names)
    df.columns = ["hash"]
    df = df.reset_index() 
    df = df.rename(columns = {'index':'file_name'})
    
    return df

In [9]:
mother_df = hash_it_up_right('C:/Projects/mother-eeg-data/*/Data/')

surf_df = hash_it_up_right('C:/Projects/EEG_explorer/Data/*/')

In [11]:
mother_df_bn = mother_df
mother_df_bn['base_name'] = mother_df_bn.file_name.apply(os.path.basename).str.lower()
mother_df_bn

,file_name,hash,base_name
0,C:/Projects/mother-eeg-data\1646832910497-Earl...,0d854854def2f7aac80df64027885d28c698ac753f664f...,015_thomas_mmn36w.cnt
1,C:/Projects/mother-eeg-data\1646832910497-Earl...,b9e53cd9807eeece442018ecbd45854e0da19f13922988...,034_17_mc_mmn36_wk.cnt
2,C:/Projects/mother-eeg-data\1646832910497-Earl...,263789094e3078e970daafa44edfa8ded6e351a2c75f7a...,036_17_mc_mmn36_wk.cnt
3,C:/Projects/mother-eeg-data\1646833329075-Earl...,e69f34a77510079aaa4763d93c3d50ab08b4430d839a9c...,036_29_mc_mmn36_wk.cnt
4,C:/Projects/mother-eeg-data\1646833329075-Earl...,05973fa3a972a3b22a4aabca66ef3bdb69a3952cf26b3b...,119_17_jr_mmn.cnt
...,...,...,...
303,C:/Projects/mother-eeg-data\1646857503973-Earl...,04b9f2317aa321750751471650247909db4d0af625990b...,608-170-29m-mc-mmn36.cnt
304,C:/Projects/mother-eeg-data\1646857661935-Earl...,cade53c907f6897f23caa787385a40cd3a17cca276d337...,602-115-17m-mc-mmn.cnt
305,C:/Projects/mother-eeg-data\1646857661935-Earl...,df70c07e3e067c2dbefc6f120f41333f2d38e4c6fa2ee3...,602-115-29m-mc-mmn36.cnt
306,C:/Projects/mother-eeg-data\1646857661935-Earl...,448a9cbc36da934efccce6ef6d184a2f3bf8e270aafa88...,604-133-17m-jc-mmn36.cnt


In [12]:
surf_df_bn = surf_df
surf_df_bn['base_name'] = surf_df_bn.file_name.apply(os.path.basename).str.lower()
surf_df_bn

,file_name,hash,base_name
0,C:/Projects/EEG_explorer/Data\11mnd mmn\001_11...,f4d67aae1412142198e6df3f1de4c79f1bdc68443431cd...,001_11_jc_mmn36_wk_mmn25_wk_mmn47_wk_mmn58_wk.cnt
1,C:/Projects/EEG_explorer/Data\11mnd mmn\003_11...,d356a1d7084d1264b76cae88a915683ebf4d5688b11198...,003_11_jc_mmn36_slp_mmn25_wk_mmn47_slp_mmn58_s...
2,C:/Projects/EEG_explorer/Data\11mnd mmn\004_11...,56b27aebacffe4eccd806cf33c175caedb9b9e6531f2a9...,004_11_mc_mmn.cnt
3,C:/Projects/EEG_explorer/Data\11mnd mmn\005_11...,3a29c74633b447d853c9a016eb7735dd6271acf177b15b...,005_11_jc_mmn2.cnt
4,C:/Projects/EEG_explorer/Data\11mnd mmn\007_11...,fc785a7147b9b8d2bc3ae9f61315065ad7ff57b7edd665...,007_11_jc_mmn2_36_wk.cnt
...,...,...,...
2317,C:/Projects/EEG_explorer/Data\5mnd mmn\740-396...,a06aecc00a8e7aa3c64364ef7fb96f442c321ab872584f...,740-396-5.5m-jr-mmn25xtr.cnt
2318,C:/Projects/EEG_explorer/Data\5mnd mmn\740-396...,c43d579aa6b917552bea9e537d89ee518514da39d90fe6...,740-396-5.5m-jr-mmn36.cnt
2319,C:/Projects/EEG_explorer/Data\5mnd mmn\740-396...,24c14ce2efb95d35b133a4e84aa7008c99eb31673ea16d...,740-396-5.5m-jr-mmn36b.cnt
2320,C:/Projects/EEG_explorer/Data\5mnd mmn\740-396...,8bfafa6c6dbd48c818937d379b53becaad4f2c8de2e0b3...,740-396-5.5m-jr-mmn47.cnt


In [13]:
results = mother_df_bn.merge(surf_df_bn, on='base_name', how='outer')

results

,file_name_x,hash_x,base_name,file_name_y,hash_y
0,C:/Projects/mother-eeg-data\1646832910497-Earl...,0d854854def2f7aac80df64027885d28c698ac753f664f...,015_thomas_mmn36w.cnt,NaN,NaN
1,C:/Projects/mother-eeg-data\1646832910497-Earl...,b9e53cd9807eeece442018ecbd45854e0da19f13922988...,034_17_mc_mmn36_wk.cnt,NaN,NaN
2,C:/Projects/mother-eeg-data\1646832910497-Earl...,263789094e3078e970daafa44edfa8ded6e351a2c75f7a...,036_17_mc_mmn36_wk.cnt,NaN,NaN
3,C:/Projects/mother-eeg-data\1646833329075-Earl...,e69f34a77510079aaa4763d93c3d50ab08b4430d839a9c...,036_29_mc_mmn36_wk.cnt,C:/Projects/EEG_explorer/Data\29mnd mmn\036_29...,20d577bc22f40881b58c09589d4be12fa1b989cee302e1...
4,C:/Projects/mother-eeg-data\1646833329075-Earl...,05973fa3a972a3b22a4aabca66ef3bdb69a3952cf26b3b...,119_17_jr_mmn.cnt,C:/Projects/EEG_explorer/Data\17mnd mmn\119_17...,0dc0f5328b3ca428f90bee0484348e3dc291e188f5a87a...
...,...,...,...,...,...
2573,NaN,NaN,740-396-5.5m-jr-mmn25xtr.cnt,C:/Projects/EEG_explorer/Data\5mnd mmn\740-396...,a06aecc00a8e7aa3c64364ef7fb96f442c321ab872584f...
2574,NaN,NaN,740-396-5.5m-jr-mmn36.cnt,C:/Projects/EEG_explorer/Data\5mnd mmn\740-396...,c43d579aa6b917552bea9e537d89ee518514da39d90fe6...
2575,NaN,NaN,740-396-5.5m-jr-mmn36b.cnt,C:/Projects/EEG_explorer/Data\5mnd mmn\740-396...,24c14ce2efb95d35b133a4e84aa7008c99eb31673ea16d...
2576,NaN,NaN,740-396-5.5m-jr-mmn47.cnt,C:/Projects/EEG_explorer/Data\5mnd mmn\740-396...,8bfafa6c6dbd48c818937d379b53becaad4f2c8de2e0b3...


In [14]:
results['same_hash'] = (results.hash_x == results.hash_y)
results

,file_name_x,hash_x,base_name,file_name_y,hash_y,same_hash
0,C:/Projects/mother-eeg-data\1646832910497-Earl...,0d854854def2f7aac80df64027885d28c698ac753f664f...,015_thomas_mmn36w.cnt,NaN,NaN,False
1,C:/Projects/mother-eeg-data\1646832910497-Earl...,b9e53cd9807eeece442018ecbd45854e0da19f13922988...,034_17_mc_mmn36_wk.cnt,NaN,NaN,False
2,C:/Projects/mother-eeg-data\1646832910497-Earl...,263789094e3078e970daafa44edfa8ded6e351a2c75f7a...,036_17_mc_mmn36_wk.cnt,NaN,NaN,False
3,C:/Projects/mother-eeg-data\1646833329075-Earl...,e69f34a77510079aaa4763d93c3d50ab08b4430d839a9c...,036_29_mc_mmn36_wk.cnt,C:/Projects/EEG_explorer/Data\29mnd mmn\036_29...,20d577bc22f40881b58c09589d4be12fa1b989cee302e1...,False
4,C:/Projects/mother-eeg-data\1646833329075-Earl...,05973fa3a972a3b22a4aabca66ef3bdb69a3952cf26b3b...,119_17_jr_mmn.cnt,C:/Projects/EEG_explorer/Data\17mnd mmn\119_17...,0dc0f5328b3ca428f90bee0484348e3dc291e188f5a87a...,False
...,...,...,...,...,...,...
2573,NaN,NaN,740-396-5.5m-jr-mmn25xtr.cnt,C:/Projects/EEG_explorer/Data\5mnd mmn\740-396...,a06aecc00a8e7aa3c64364ef7fb96f442c321ab872584f...,False
2574,NaN,NaN,740-396-5.5m-jr-mmn36.cnt,C:/Projects/EEG_explorer/Data\5mnd mmn\740-396...,c43d579aa6b917552bea9e537d89ee518514da39d90fe6...,False
2575,NaN,NaN,740-396-5.5m-jr-mmn36b.cnt,C:/Projects/EEG_explorer/Data\5mnd mmn\740-396...,24c14ce2efb95d35b133a4e84aa7008c99eb31673ea16d...,False
2576,NaN,NaN,740-396-5.5m-jr-mmn47.cnt,C:/Projects/EEG_explorer/Data\5mnd mmn\740-396...,8bfafa6c6dbd48c818937d379b53becaad4f2c8de2e0b3...,False


In [15]:
common_files_at_zero

{'036_29_mc_mmn36_wk.cnt',
 '119_17_jr_mmn.cnt',
 '119_29_jr_mmn36_wk.cnt',
 '133_17_jr_mmn.cnt',
 '133_29_jr_mmn36_wk.cnt',
 '158_17_mr_mmn36.cnt',
 '158_29_mr_mmn36_wk.cnt',
 '162_17_mr_mmn36_wk.cnt',
 '164_17_jr_mmn36_wk.cnt',
 '164_29_jr_mmn36_wk.cnt',
 '172_17_mr_mmn36.cnt',
 '172_29_mr_mmn36_wk.cnt',
 '179_29_mr_mmn36_wk.cnt',
 '333_29_jc_mmn36_wk.cnt',
 '489_29_md_mmn36_wk.cnt',
 '604-133-29m-jc-mmn36.cnt',
 '605-131-29m-jc-mmn36.cnt',
 '607-128-29m-jc-mmn36.cnt',
 '608-170-29m-mc-mmn36.cnt',
 '609-158-29m-jc-mmn36.cnt',
 '610-185-29m-jc-mmn36-2-waak.cnt',
 '613-176-29m-mc-mmn36.cnt',
 '618-163-29m-jc-mmn36.cnt',
 '622-189-29m-jc-mmn36.cnt',
 '623-321-29m-mc-mmn36.cnt',
 '625-253-29m-jc-mmn36.cnt',
 '628-369-29m-mc-mmn36.cnt',
 '629-357-29m-jc-mmn36.cnt',
 '635-364-29m-mc-mmn36.cnt',
 '640-464-29m-jc-mmn36.cnt',
 '642-485-29m-jc-mmn36.cnt',
 '646-478-29m-mc-mmn36.cnt',
 '707-060-29m-jr-mmn36-waak.cnt',
 '719-079-29m-jr-mmn36.cnt',
 '724-116-29m-jr-mmn36.cnt',
 '725-161-29m-jr-mm

In [19]:
filter_to_same_name = results[results['base_name'].isin(common_files_at_zero)]
filter_to_same_name 

,file_name_x,hash_x,base_name,file_name_y,hash_y,same_hash
3,C:/Projects/mother-eeg-data\1646833329075-Earl...,e69f34a77510079aaa4763d93c3d50ab08b4430d839a9c...,036_29_mc_mmn36_wk.cnt,C:/Projects/EEG_explorer/Data\29mnd mmn\036_29...,20d577bc22f40881b58c09589d4be12fa1b989cee302e1...,False
4,C:/Projects/mother-eeg-data\1646833329075-Earl...,05973fa3a972a3b22a4aabca66ef3bdb69a3952cf26b3b...,119_17_jr_mmn.cnt,C:/Projects/EEG_explorer/Data\17mnd mmn\119_17...,0dc0f5328b3ca428f90bee0484348e3dc291e188f5a87a...,False
5,C:/Projects/mother-eeg-data\1646833329075-Earl...,4930fa5b0f4af8c331d97dde6e25da8db769380cedfc10...,119_29_jr_mmn36_wk.cnt,C:/Projects/EEG_explorer/Data\29mnd mmn\119_29...,eb5ae61c5b0b4f757d31aaa85077397db13b587904c226...,False
6,C:/Projects/mother-eeg-data\1646833329075-Earl...,2a598930388709df7e0221364c2e466e6e4df348e5ace7...,133_17_jr_mmn.cnt,C:/Projects/EEG_explorer/Data\17mnd mmn\133_17...,fde399d9820f6067babde6e7fd5007b881992dbb8bcf9a...,False
7,C:/Projects/mother-eeg-data\1646833396252-Earl...,e8e0cd0b0cba56c341e7ec1ecb5d6f9029f87afc61f0c7...,133_29_jr_mmn36_wk.cnt,C:/Projects/EEG_explorer/Data\29mnd mmn\133_29...,ac740e8da325ab25d08ad57869cd711c1b17af996bc92a...,False
...,...,...,...,...,...,...
295,C:/Projects/mother-eeg-data\1646854865829-Earl...,f7aca311047979f5a17290aebafea9c54755019654b282...,628-369-29m-mc-mmn36.cnt,C:/Projects/EEG_explorer/Data\29mnd mmn\628-36...,361e2700c431671176e1d944c3d8fd7f17cd5a080de923...,False
300,C:/Projects/mother-eeg-data\1646855777982-Earl...,fba762d12f6c780617ecacca0ae0bb560057357160c3e0...,629-357-29m-jc-mmn36.cnt,C:/Projects/EEG_explorer/Data\29mnd mmn\629-35...,1ac492adc8bf6d068caa56237ef46116f1c9d8a640a4ee...,False
301,C:/Projects/mother-eeg-data\1646855857455-Earl...,41370e381911890f729f6d38d80ddf600d4221d3439eee...,629-357-29m-jc-mmn36.cnt,C:/Projects/EEG_explorer/Data\29mnd mmn\629-35...,1ac492adc8bf6d068caa56237ef46116f1c9d8a640a4ee...,False
302,C:/Projects/mother-eeg-data\1646855952576-Earl...,4a995b234b5e46cc5766892b04b2cc523810108d972f76...,629-357-29m-jc-mmn36.cnt,C:/Projects/EEG_explorer/Data\29mnd mmn\629-35...,1ac492adc8bf6d068caa56237ef46116f1c9d8a640a4ee...,False


In [22]:
filter_to_same_name[filter_to_same_name['same_hash']]

,file_name_x,hash_x,base_name,file_name_y,hash_y,same_hash


## OK, let's start looking one by one:

WE will choose the file 036_29_mc_mmn36_wk.cnt for our first file to look inside. 

In [37]:
print(filter_to_same_name.iloc[0,3])

C:/Projects/EEG_explorer/Data\29mnd mmn\036_29_mc_mmn36_wk.cnt


In [39]:
zero_36_29_mc_mmn36_wk_file_from_mother = 'C:/Projects/mother-eeg-data/1646833329075-Early_Precursors_of_Famil/Data/036_29_mc_mmn36_wk.cnt'

zero_36_29_mc_mmn36_wk_file_from_surf = 'C:/Projects/EEG_explorer/Data/29mnd mmn/036_29_mc_mmn36_wk.cnt'

In [40]:

# read files 
mother_data_raw = mne.io.read_raw_cnt(zero_36_29_mc_mmn36_wk_file_from_mother, eog='auto', preload=True)
surf_data_raw = mne.io.read_raw_cnt(zero_36_29_mc_mmn36_wk_file_from_surf, eog='auto', preload=True)

Reading 0 ... 778159  =      0.000 ...  1556.318 secs...
Reading 0 ... 778159  =      0.000 ...  1556.318 secs...


In [47]:
print('Data types(mother): {}\n\n{}\n'.format(type(mother_data_raw), mother_data_raw))
print('Data types(surf): {}\n\n{}\n'.format(type(surf_data_raw), surf_data_raw))

# Get the sample rate
print('Sample rate(mother):', mother_data_raw.info['sfreq'], 'Hz')
print('Sample rate(surf):', surf_data_raw.info['sfreq'], 'Hz')

# Get the size of the matrix
print('Size of the matrix(mother): {}\n'.format(mother_data_raw.get_data().shape))
print('Size of the matrix(surf): {}\n'.format(surf_data_raw.get_data().shape))

# The mne.info class can be used to learn more about the data.
print(surf_data_raw.info)

Data types(mother): <class 'mne.io.cnt.cnt.RawCNT'>

<RawCNT | 036_29_mc_mmn36_wk.cnt, 64 x 778160 (1556.3 s), ~380.1 MB, data loaded>

Data types(surf): <class 'mne.io.cnt.cnt.RawCNT'>

<RawCNT | 036_29_mc_mmn36_wk.cnt, 64 x 778160 (1556.3 s), ~380.1 MB, data loaded>

Sample rate(mother): 500.0 Hz
Sample rate(surf): 500.0 Hz
Size of the matrix(mother): (64, 778160)

Size of the matrix(surf): (64, 778160)

<Info | 8 non-empty values
 bads: []
 ch_names: O2, O1, OZ, PZ, P4, CP4, P8, C4, TP8, T8, P7, P3, CP3, CPZ, CZ, ...
 chs: 62 EEG, 2 EOG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: 2004-04-08 13:16:41 UTC
 nchan: 64
 projs: []
 sfreq: 500.0 Hz
 subject_info: 5 items (dict)
>


In [48]:
print(mother_data_raw.info)

<Info | 8 non-empty values
 bads: []
 ch_names: O2, O1, OZ, PZ, P4, CP4, P8, C4, TP8, T8, P7, P3, CP3, CPZ, CZ, ...
 chs: 62 EEG, 2 EOG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: 2004-04-08 13:16:41 UTC
 nchan: 64
 projs: []
 sfreq: 500.0 Hz
 subject_info: 5 items (dict)
>


In [51]:
surf_raw_df = surf_data_raw.to_data_frame()
mother_raw_df = mother_data_raw.to_data_frame()

surf_raw_df.head()

,time,O2,O1,OZ,PZ,P4,CP4,P8,C4,TP8,...,F2,F6,FC5,F1,AF4,AF8,F5,AF7,AF3,FPZ
0,0,-249.325775,-117.523973,-296.347844,-256.544505,-167.551096,-185.763898,-128.937095,-121.580550,-53.425842,...,-324.906120,30.550920,-178.934111,-292.010083,-472.681693,-426.934324,84.689997,-169.405812,-46.851388,-208.187129
1,2,-242.540217,-113.164775,-288.173931,-253.403144,-165.463232,-181.944453,-127.548809,-118.627880,-52.905460,...,-322.903617,25.709517,-172.052030,-288.645911,-469.841258,-428.439970,85.196112,-169.405812,-42.149456,-213.700914
2,4,-237.668534,-108.456841,-282.260887,-250.610823,-163.027391,-179.861120,-124.425164,-116.891015,-51.344315,...,-321.568615,22.370619,-167.855639,-286.290991,-466.833740,-429.276441,86.883164,-166.034552,-38.958860,-213.032576
3,6,-235.928647,-103.400172,-279.478278,-248.342062,-160.939527,-179.340286,-120.780913,-116.369955,-50.477013,...,-320.901114,19.198665,-167.016361,-285.618157,-464.661643,-428.439970,89.076331,-162.157603,-38.790934,-206.516286
4,8,-234.188760,-98.692238,-277.391321,-245.898781,-157.981719,-178.645842,-117.310197,-115.501523,-51.517776,...,-319.732987,14.691153,-164.834238,-284.104280,-463.157883,-424.759502,91.269499,-153.223764,-39.294712,-199.832911


In [52]:
mother_raw_df.head()

,time,O2,O1,OZ,PZ,P4,CP4,P8,C4,TP8,...,F2,F6,FC5,F1,AF4,AF8,F5,AF7,AF3,FPZ
0,0,-249.325775,-117.523973,-296.347844,-256.544505,-167.551096,-185.763898,-128.937095,-121.580550,-53.425842,...,-324.906120,30.550920,-178.934111,-292.010083,-472.681693,-426.934324,84.689997,-169.405812,-46.851388,-208.187129
1,2,-242.540217,-113.164775,-288.173931,-253.403144,-165.463232,-181.944453,-127.548809,-118.627880,-52.905460,...,-322.903617,25.709517,-172.052030,-288.645911,-469.841258,-428.439970,85.196112,-169.405812,-42.149456,-213.700914
2,4,-237.668534,-108.456841,-282.260887,-250.610823,-163.027391,-179.861120,-124.425164,-116.891015,-51.344315,...,-321.568615,22.370619,-167.855639,-286.290991,-466.833740,-429.276441,86.883164,-166.034552,-38.958860,-213.032576
3,6,-235.928647,-103.400172,-279.478278,-248.342062,-160.939527,-179.340286,-120.780913,-116.369955,-50.477013,...,-320.901114,19.198665,-167.016361,-285.618157,-464.661643,-428.439970,89.076331,-162.157603,-38.790934,-206.516286
4,8,-234.188760,-98.692238,-277.391321,-245.898781,-157.981719,-178.645842,-117.310197,-115.501523,-51.517776,...,-319.732987,14.691153,-164.834238,-284.104280,-463.157883,-424.759502,91.269499,-153.223764,-39.294712,-199.832911


In [53]:
mother_raw_df.equals(surf_raw_df)

True